# Model

##  Importing libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Mounted at /content/drive


In [2]:
! pip install skimpy
! pip install category_encoders
! pip install xgboost
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 51.2 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.4 which is incompatib

In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from skimpy import skim
import warnings
import os
import pickle
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from catboost import CatBoostClassifier
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## Load the data

In [4]:
training_encoder = pd.read_csv("/content/drive/MyDrive/WIDS/DATAS/training_encoder.csv")

In [5]:
skim(training_encoder)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 12906  │ │ float64     │ 79    │                                                          │
│ │ Number of columns │ 83     │ │ int64       │ 4     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %  ┃ mean   ┃ sd     ┃ p0     ┃ p25    ┃ p50    ┃ p75    ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ patient_id         │    0 │     0 │ 550000 │ 260000 │ 100000 │ 320000 │ 540000 │ 770000 │ 1000000 │ ▇▇▇▇▇▇ │  │
│ │ patient_race       │    0 │     0 │   0.62 │  0.033 │   0.21 │    0.6 │   0.61 │   0.65 │    0.92 │     ▇  │  │
│ │ payer_type         │    0 │     0 │   0.62 │   0.04 │   0.12 │    0.6 │   0.61 │   0.63 │    0.91 │    ▇▁  │  │
│ │ patient_state      │    0 │     0 │   0.62 │  0.064 │   0.12 │   0.58 │   0.63 │   0.65 │    0.94 │    ▇▁  │  │
│ │ patient_gender     │    0 │     0 │   0.62 │  0.011 │   0.45 │   0.62 │   0.62 │   0.63 │    0.91 │    ▇   │  │
│ │ breast_cancer_diag │    0 │     0 │   0.63 │   0.29 │  0.026 │   0.66 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ nosis_code         │      │       │        │        │        │        │        │        │         │        │  │
│ │ breast_cancer_diag │    0 │     0 │   0.63 │   0.29 │  0.026 │   0.66 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ nosis_desc         │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_cancer_ │    0 │     0 │   0.62 │  0.076 │   0.12 │   0.56 │   0.67 │   0.67 │    0.96 │   ▁▇▁  │  │
│ │ diagnosis_code     │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_first_n │    0 │     0 │   0.62 │  0.012 │   0.21 │   0.62 │   0.62 │   0.63 │    0.91 │     ▇  │  │
│ │ ovel_treatment     │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_first_n │    0 │     0 │   0.62 │  0.012 │   0.21 │   0.62 │   0.62 │   0.63 │    0.91 │     ▇  │  │
│ │ ovel_treatment_typ │      │       │        │        │        │        │        │        │         │        │  │
│ │ e                  │      │       │        │        │        │        │        │        │         │        │  │
│ │ Region             │    0 │     0 │   0.62 │  0.033 │   0.12 │    0.6 │   0.63 │   0.65 │    0.87 │    ▆▇  │  │
│ │ Division           │    0 │     0 │   0.62 │  0.039 │   0.12 │    0.6 │   0.63 │   0.64 │    0.87 │    ▆▇  │  │
│ │ patient_zip3       │    0 │     0 │    570 │    280 │    100 │    330 │    550 │    850 │    1000 │ ▅▅▆▃▅▇ │  │
│ │ patient_age        │    0 │     0 │     59 │     13 │     18 │     50 │     59 │     67 │      91 │  ▂▅▇▃▂ │  │
│ │ bmi                │ 8965 │ 69.46 │     29 │    5.7 │     14 │     25 │     28 │     33 │      85 │  ▅▇▁   │  │
│ │ population         │    1 │  0.01 │  21000 │  14000 │    640 │   9500 │  19000 │  30000 │   71000 │ ▇▇▅▂▁  │  │
│ │ density            │    1 │  0.01 │   1600 │   3000 

## Pre-Model

In [6]:
# Obtener X y Y
X = training_encoder.drop(['DiagPeriodL90D','patient_id'],axis = 1)
Y = training_encoder['DiagPeriodL90D']

In [7]:
# Divir la data en Train y Test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state =42)

In [8]:
#Para Gradient Boosting Classifier
X_train_fillna = X_train.fillna(0)
X_test_fillna = X_test.fillna(0)

In [9]:
skim(X_train) # Con nulos

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 9034   │ │ float64     │ 79    │                                                          │
│ │ Number of columns │ 81     │ │ int64       │ 2     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %  ┃ mean   ┃ sd     ┃ p0     ┃ p25    ┃ p50    ┃ p75    ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ patient_race       │    0 │     0 │   0.62 │  0.032 │   0.31 │    0.6 │   0.61 │   0.65 │    0.91 │    ▇▇  │  │
│ │ payer_type         │    0 │     0 │   0.62 │  0.039 │   0.16 │    0.6 │   0.61 │   0.63 │    0.91 │    ▇▁  │  │
│ │ patient_state      │    0 │     0 │   0.62 │  0.063 │   0.16 │   0.58 │   0.63 │   0.65 │    0.94 │    ▇▁  │  │
│ │ patient_gender     │    0 │     0 │   0.62 │   0.01 │   0.47 │   0.62 │   0.62 │   0.63 │    0.87 │    ▇   │  │
│ │ breast_cancer_diag │    0 │     0 │   0.63 │   0.29 │  0.026 │   0.68 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ nosis_code         │      │       │        │        │        │        │        │        │         │        │  │
│ │ breast_cancer_diag │    0 │     0 │   0.63 │   0.29 │  0.026 │   0.68 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ nosis_desc         │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_cancer_ │    0 │     0 │   0.62 │  0.077 │   0.12 │   0.56 │   0.67 │   0.67 │    0.96 │   ▁▇▁  │  │
│ │ diagnosis_code     │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_first_n │    0 │     0 │   0.62 │  0.012 │   0.21 │   0.62 │   0.62 │   0.63 │    0.87 │     ▇  │  │
│ │ ovel_treatment     │      │       │        │        │        │        │        │        │         │        │  │
│ │ metastatic_first_n │    0 │     0 │   0.62 │  0.011 │   0.21 │   0.62 │   0.62 │   0.63 │    0.87 │     ▇  │  │
│ │ ovel_treatment_typ │      │       │        │        │        │        │        │        │         │        │  │
│ │ e                  │      │       │        │        │        │        │        │        │         │        │  │
│ │ Region             │    0 │     0 │   0.62 │  0.032 │   0.16 │    0.6 │   0.63 │   0.65 │    0.87 │    ▇▇  │  │
│ │ Division           │    0 │     0 │   0.62 │  0.038 │   0.16 │    0.6 │   0.63 │   0.64 │    0.87 │    ▇▃  │  │
│ │ patient_zip3       │    0 │     0 │    570 │    280 │    100 │    330 │    560 │    850 │    1000 │ ▅▅▆▃▅▇ │  │
│ │ patient_age        │    0 │     0 │     59 │     13 │     18 │     50 │     59 │     67 │      91 │  ▂▅▇▃▂ │  │
│ │ bmi                │ 6267 │ 69.37 │     29 │    5.8 │     14 │     25 │     28 │     33 │      85 │  ▅▇▁   │  │
│ │ population         │    1 │  0.01 │  21000 │  14000 │    640 │   9500 │  19000 │  30000 │   71000 │ ▇▇▅▂▁  │  │
│ │ density            │    1 │  0.01 │   1600 │   2900 │   0.92 │    170 │    700 │   1700 │   21000 │   ▇▁   │  │
│ │ age_median         │    1 │  0.01 │     41 │    4.1 

In [10]:
skim(X_train_fillna) #sin nulos

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 9034   │ │ float64     │ 79    │                                                          │
│ │ Number of columns │ 81     │ │ int64       │ 2     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name           ┃ NA  ┃ NA % ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p50    ┃ p75    ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ patient_race          │   0 │    0 │   0.62 │  0.032 │  0.31 │    0.6 │   0.61 │   0.65 │    0.91 │    ▇▇  │  │
│ │ payer_type            │   0 │    0 │   0.62 │  0.039 │  0.16 │    0.6 │   0.61 │   0.63 │    0.91 │    ▇▁  │  │
│ │ patient_state         │   0 │    0 │   0.62 │  0.063 │  0.16 │   0.58 │   0.63 │   0.65 │    0.94 │    ▇▁  │  │
│ │ patient_gender        │   0 │    0 │   0.62 │   0.01 │  0.47 │   0.62 │   0.62 │   0.63 │    0.87 │    ▇   │  │
│ │ breast_cancer_diagnos │   0 │    0 │   0.63 │   0.29 │ 0.026 │   0.68 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ is_code               │     │      │        │        │       │        │        │        │         │        │  │
│ │ breast_cancer_diagnos │   0 │    0 │   0.63 │   0.29 │ 0.026 │   0.68 │   0.78 │   0.79 │    0.98 │ ▃   ▇▁ │  │
│ │ is_desc               │     │      │        │        │       │        │        │        │         │        │  │
│ │ metastatic_cancer_dia │   0 │    0 │   0.62 │  0.077 │  0.12 │   0.56 │   0.67 │   0.67 │    0.96 │   ▁▇▁  │  │
│ │ gnosis_code           │     │      │        │        │       │        │        │        │         │        │  │
│ │ metastatic_first_nove │   0 │    0 │   0.62 │  0.012 │  0.21 │   0.62 │   0.62 │   0.63 │    0.87 │     ▇  │  │
│ │ l_treatment           │     │      │        │        │       │        │        │        │         │        │  │
│ │ metastatic_first_nove │   0 │    0 │   0.62 │  0.011 │  0.21 │   0.62 │   0.62 │   0.63 │    0.87 │     ▇  │  │
│ │ l_treatment_type      │     │      │        │        │       │        │        │        │         │        │  │
│ │ Region                │   0 │    0 │   0.62 │  0.032 │  0.16 │    0.6 │   0.63 │   0.65 │    0.87 │    ▇▇  │  │
│ │ Division              │   0 │    0 │   0.62 │  0.038 │  0.16 │    0.6 │   0.63 │   0.64 │    0.87 │    ▇▃  │  │
│ │ patient_zip3          │   0 │    0 │    570 │    280 │   100 │    330 │    560 │    850 │    1000 │ ▅▅▆▃▅▇ │  │
│ │ patient_age           │   0 │    0 │     59 │     13 │    18 │     50 │     59 │     67 │      91 │  ▂▅▇▃▂ │  │
│ │ bmi                   │   0 │    0 │    8.9 │     14 │     0 │      0 │      0 │     24 │      85 │  ▇▂▂   │  │
│ │ population            │   0 │    0 │  21000 │  14000 │     0 │   9500 │  19000 │  30000 │   71000 │ ▇▇▅▂▁  │  │
│ │ density               │   0 │    0 │   1600 │   2900 │     0 │    170 │    700 │   1700 │   21000 │   ▇▁   │  │
│ │ age_median            │   0 │    0 │     41 │    4.1 │     0 │     37 │     41 │     43 │      55 │    ▂▇▁ │  │
│ │ age_under_10          │   0 │    0 │     11 │    1.5

## Gradient Boosting Classifier

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate= 0.1 ,
                                loss='exponential' ,
                                n_estimators=150 ,
                                random_state=42)
gbc.fit(X_train_fillna, Y_train)

GradientBoostingClassifier(loss='exponential', n_estimators=150,
                           random_state=42)

In [12]:
# Guardar Modelos
import os
import pickle
pickle.dump(gbc, open(os.path.join('/content/drive/MyDrive/WIDS/MODELS/GradientBoostingClassifier.sav'), 'wb'))
print('Se ha guardado exitosamente')

Se ha guardado exitosamente


In [13]:
Y_pred = gbc.predict(X_test_fillna)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
# accuracy score
gbc_acc = accuracy_score(Y_test, Y_pred)
print(gbc_acc)

0.8145661157024794


In [16]:
# confusion matrix
print(confusion_matrix(Y_test, Y_pred))

[[ 844  616]
 [ 102 2310]]


In [17]:
# classification report
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.58      0.70      1460
           1       0.79      0.96      0.87      2412

    accuracy                           0.81      3872
   macro avg       0.84      0.77      0.78      3872
weighted avg       0.83      0.81      0.80      3872



## XGBOOST Classifier

In [18]:
import xgboost as xgb
xgbC = xgb.XGBClassifier(
    alpha=0,
    reg_lambda=1,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=150,
    random_state=42
)
xgbC.fit(X_train, Y_train)

XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, ...)

In [19]:
# Guardar Modelos
import os
import pickle
pickle.dump(xgbC, open(os.path.join('/content/drive/MyDrive/WIDS/MODELS/XGBOOSTClassifer.sav'), 'wb'))
print('Se ha guardado exitosamente')

Se ha guardado exitosamente


In [20]:
Y_pred_xgbc = xgbC.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [22]:
# accuracy score
xgbC_acc = accuracy_score(Y_test, Y_pred_xgbc)
print(xgbC_acc)

0.8150826446280992


In [23]:
# confusion matrix
print(confusion_matrix(Y_test, Y_pred_xgbc))

[[ 842  618]
 [  98 2314]]


In [24]:
# classification report
print(classification_report(Y_test, Y_pred_xgbc))

              precision    recall  f1-score   support

           0       0.90      0.58      0.70      1460
           1       0.79      0.96      0.87      2412

    accuracy                           0.82      3872
   macro avg       0.84      0.77      0.78      3872
weighted avg       0.83      0.82      0.80      3872



## CatBoostClassifier

In [25]:
from catboost import Pool, CatBoostClassifier
train_pool = Pool(X_train, Y_train)
cbc = CatBoostClassifier(depth =  4 ,
                        iterations = 200 ,
                        l2_leaf_reg=1 ,
                        learning_rate =  0.01,
                        verbose=True ,
                        random_state=42)
cbc.fit(train_pool)

0:	learn: 0.6884072	total: 60.3ms	remaining: 12s
1:	learn: 0.6837553	total: 72.8ms	remaining: 7.21s
2:	learn: 0.6792041	total: 86.6ms	remaining: 5.69s
3:	learn: 0.6748377	total: 99.4ms	remaining: 4.87s
4:	learn: 0.6705773	total: 115ms	remaining: 4.49s
5:	learn: 0.6662979	total: 125ms	remaining: 4.05s
6:	learn: 0.6622538	total: 136ms	remaining: 3.76s
7:	learn: 0.6581782	total: 147ms	remaining: 3.53s
8:	learn: 0.6542791	total: 158ms	remaining: 3.34s
9:	learn: 0.6503871	total: 168ms	remaining: 3.19s
10:	learn: 0.6465533	total: 178ms	remaining: 3.06s
11:	learn: 0.6427740	total: 188ms	remaining: 2.94s
12:	learn: 0.6391470	total: 199ms	remaining: 2.86s
13:	learn: 0.6357654	total: 209ms	remaining: 2.78s
14:	learn: 0.6322723	total: 219ms	remaining: 2.7s
15:	learn: 0.6290497	total: 229ms	remaining: 2.64s
16:	learn: 0.6258821	total: 240ms	remaining: 2.58s
17:	learn: 0.6226145	total: 250ms	remaining: 2.53s
18:	learn: 0.6193908	total: 265ms	remaining: 2.53s
19:	learn: 0.6162584	total: 276ms	remain

In [26]:
# Guardar Modelos
import os
import pickle
pickle.dump(cbc, open(os.path.join('/content/drive/MyDrive/WIDS/MODELS/CatBoostClassifier.sav'), 'wb'))
print('Se ha guardado exitosamente')

Se ha guardado exitosamente


In [27]:
Y_pred_cbc = cbc.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [29]:
# accuracy score
cbc_acc = accuracy_score(Y_test, Y_pred_cbc)
print(cbc_acc)

0.8163739669421488


In [30]:
# confusion matrix
print(confusion_matrix(Y_test, Y_pred_cbc))

[[ 837  623]
 [  88 2324]]


In [31]:
# classification report
print(classification_report(Y_test, Y_pred_cbc))

              precision    recall  f1-score   support

           0       0.90      0.57      0.70      1460
           1       0.79      0.96      0.87      2412

    accuracy                           0.82      3872
   macro avg       0.85      0.77      0.78      3872
weighted avg       0.83      0.82      0.80      3872

